In [ ]:
# Tell python where to look for modules. 
# Depending on how your jupyter handles working directories, this may not be needed.
import sys
sys.path.append('../../hourly-egrid/')

# import the necessary packages
%reload_ext autoreload
%autoreload 2

# import packages
import os
import requests
import tarfile
import sqlalchemy as sa
from pathlib import Path
import pandas as pd
import plotly.express as px
import numpy as np

import src.data_cleaning as data_cleaning
import src.gross_to_net_generation as gross_to_net_generation
import src.load_data as load_data
import src.distribute_eia923 as distribute_eia923

In [ ]:
# specify the year for which you want data

year = 2020

## Download the data

In [ ]:
############### PUDL Database ######################

load_data.download_pudl_data(zenodo_url = 'https://zenodo.org/record/6349861/files/pudl-v0.6.0-2022-03-12.tgz')

################# eGRID data #########################

# the 2019 and 2020 data appear to be hosted on different urls
egrid_files_to_download = ['https://www.epa.gov/sites/default/files/2021-02/egrid2019_data.xlsx', 
                           'https://www.epa.gov/system/files/documents/2022-01/egrid2020_data.xlsx']

load_data.download_egrid_files(egrid_files_to_download)

############# EIA-930 data #####################

load_data.download_eia930_data(years_to_download=[year])

########## Power Sector Data Crosswalk #############
# NOTE: Check for new releases at https://github.com/USEPA/camd-eia-crosswalk

load_data.download_epa_psdc(psdc_url='https://github.com/USEPA/camd-eia-crosswalk/releases/download/v0.2.1/epa_eia_crosswalk.csv')


## Load and clean the cems data

In [ ]:
# clean generation and fuel data from EIA-923
gen_fuel_allocated = distribute_eia923.allocate_gen_fuel_by_gen(year=year)

# clean the cems data
cems = data_cleaning.clean_cems(year)

# flag any generator-months for which we already have cems data
gen_fuel_allocated = data_cleaning.identify_emissions_data_source(cems, gen_fuel_allocated)

# convert hourly gross generation to net generation
cems = data_cleaning.convert_gross_to_net_generation(cems, gen_fuel_allocated)

# for generators where there is heat input but no gross generation reported, impute hourly net generation based on reported EIA values
cems = data_cleaning.impute_missing_hourly_net_generation(cems, gen_fuel_allocated)

# add information about the balancing authority 
cems = data_cleaning.assign_ba_code_to_plant(cems, year)

## Output data to CSV

In [ ]:
# output the cems data to csv for others to use (update the date)
# removes any hourly data where zero data is reported in order to keep the file size smaller
cems_for_export = cems.loc[cems[['co2_mass_tons','gross_generation_mwh','heat_content_mmbtu']].sum() > 0, ['plant_id_eia', 'unitid', 'operating_datetime_utc',
       'gross_generation_mwh','net_generation_mwh','steam_load_1000_lbs', 'heat_content_mmbtu', 'co2_mass_tons',
       'report_date', 'cems_reporting_category', 'energy_source_code',
       'ba_code', 'state']]

date = '20220415'
cems_for_export.to_csv(f'../data/output/cems_{year}_cleaned_{date}.csv')